In [21]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import distance_helpers
import numpy as np
import json
import cv2
import operator

In [22]:
inputfiles = ['petfinderPurebred_pca_11063_100.pickle','petfinderPurebred_dog_cnn_simple.pickle','petfinderPurebred_dog_cnn_inception.pickle']

In [23]:
photo_ids = []
breedList = dict()
photoId2Index = dict()
with open("small_dog_data.json") as data_file:    
    dog_data = json.load(data_file)
counter = 0
for k,v in dog_data.iteritems():
    nBreeds = float(len(v["breeds"]))
    if len(v["breeds"]) > 1: continue
    breed = v["breeds"][0]
    photos = v["photos"]
    for photo in photos:
        imagename = "cutPetFinder/"+k+"_"+photo["id"]+".jpg"
        img = cv2.imread(imagename)
        if img is None: continue
        photo_ids.append(k)
        photoId2Index[k] = counter
        if not breedList.has_key(breed): breedList[breed] = []
        breedList[breed].append(counter)
        counter += 1
        break

In [24]:
print len(photo_ids),len(breedList)

1541 111


In [31]:
breedDistances = { key : [] for key in breedList.keys()}
sumBreedDistances = [0 for i in range(len(inputfiles))]
breedMedians = { key : [] for key in breedList.keys()}

for i,inputfilename in enumerate(inputfiles):
    with open(inputfilename, 'rb') as handle:
        b = pickle.load(handle)
        coefficients = np.array(b.values())
        similarities = distance_helpers.getDistances(coefficients)
        avgForAll = distance_helpers.getAvgDistance(similarities)
        medForAll = np.median(similarities)
        print inputfilename,coefficients.shape,avgForAll,medForAll
        for k,v in breedList.iteritems():
            #print k,len(v)
            if len(v) < 20:
                breedDistances[k].append(0)
                continue
            idx = np.array(v)
            slimSimilarities = similarities[idx[:, None], idx]
            breedAvg = distance_helpers.getAvgDistance(slimSimilarities)
            breedMedian = np.median(slimSimilarities)
            breedMedians[k].append(breedMedian)
            breedDistances[k].append(breedAvg/avgForAll)
            sumBreedDistances[i] += breedAvg/avgForAll

petfinderPurebred_pca_11063_100.pickle (1541, 100) 0.0930056695018 0.18590056721
petfinderPurebred_dog_cnn_simple.pickle (1541, 120) 0.00898506303884 4.69937e-08
petfinderPurebred_dog_cnn_inception.pickle (1541, 120) 0.0479053595658 0.0148531


In [26]:
print sumBreedDistances

[13.536538662564611, 13.402936305429302, 12.672095056421453]


In [32]:
skimBreedDistances = dict()
for k,l in breedMedians.iteritems():
    if sum(l) == 0: continue
    skimBreedDistances[k] = l[2]
print skimBreedDistances

{u'Rottweiler': 0.017997444, u'German Shepherd Dog': 0.02131453, u'Chihuahua': 0.01768145, u'Shepherd': 0.018534452, u'Shih Tzu': 0.013821457, u'American Staffordshire Terrier': 0.016110372, u'Poodle': 0.00571323, u'Siberian Husky': 0.013458723, u'Boxer': 0.010805424, u'Pit Bull Terrier': 0.016720675, u'Australian Cattle Dog (Blue Heeler)': 0.014413328, u'Labrador Retriever': 0.016272394, u'Terrier': 0.017972868, u'Dachshund': 0.0081638182}


In [33]:
sortedDistances = sorted(skimBreedDistances.items(), key=operator.itemgetter(1))
print [(k,v,len(breedList[k])) for k,v in sortedDistances]

[(u'Poodle', 0.00571323, 30), (u'Dachshund', 0.0081638182, 27), (u'Boxer', 0.010805424, 76), (u'Siberian Husky', 0.013458723, 29), (u'Shih Tzu', 0.013821457, 23), (u'Australian Cattle Dog (Blue Heeler)', 0.014413328, 25), (u'American Staffordshire Terrier', 0.016110372, 26), (u'Labrador Retriever', 0.016272394, 53), (u'Pit Bull Terrier', 0.016720675, 278), (u'Chihuahua', 0.01768145, 281), (u'Terrier', 0.017972868, 126), (u'Rottweiler', 0.017997444, 33), (u'Shepherd', 0.018534452, 32), (u'German Shepherd Dog', 0.02131453, 97)]


In [29]:
print len(sortedDistances)

14
